## Imports


In order for us to read the files, we need to import certain modules. Pandas and altair allow us to plot the data in the picologger file

In [1]:
import os
import sys
import pandas as pd
import altair as alt
from datetime import date

In [2]:
### package versions
packList=["pandas","altair","numpy","itkdb"]
for p in packList:
    if p in sys.modules.keys():
        mod=sys.modules.get(p)
        print(mod.__name__+" : "+mod.__version__)

pandas : 1.4.4
altair : 5.0.1
numpy : 1.21.5


## Read in Data

In [12]:
### list files in above directory
cwd = os.getcwd()
dataDir="C:/Users/User/Documents/GitHub/picoLogger/"
os.listdir(dataDir)

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'exampleFile.txt',
 'notebooks',
 'output_file.csv',
 'picolog_folder',
 'README.md',
 'requirements.txt']

## Format raw data
The following data takes in all of the files from the picologger folder,performs necessary calculations on them and puts them all in one csv file.

In [13]:
def perform_calculations(input_folder, output_file):
    all_data = []
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            data = pd.read_csv(file_path)
            
            # Calculate Vin Drop and GND Drop
            data['Vin Drop (V)'] = data['Vin+ Last (V)'] - data['Vin- Last (V)']
            data['GND Drop (V)'] = data['GND+ Last (V)'] - data['GND- Last (V)']
            
            # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
            data['Resistance Vin(Ohms)'] = data['Vin Drop (V)'] / 10 / 5
            data['Resistance GND(Ohms)'] = data['GND Drop (V)'] / 10 / 5
            data['Total Resistance(mOhms)'] = (data['Resistance Vin(Ohms)'] + data['Resistance GND(Ohms)']) * 1000
            
            # Calculate Capacitor Equivalent leakage current(nA)
            data['Capacitor Equivalent leakage current(nA)'] = ((data['Capacitor leakage test Last (V)'] / 10) / (1 * 10 ** 6)) / 1000000
            
            # Calculate NTC value(Kohms)
            data['NTC value (Kohms)'] = 0.2 * 51 / data['NTC Last (V)']
            
            # Prepare the new DataFrame with desired columns and values
            result_data = pd.DataFrame({
                'component': filename,
                'componentType': 'PCB',
                'stage': 'PCB_QC',
                'testType': 'HV_LV_TEST',
                'institution': 'Glasgow',
                'runNumber': data.shape[0],
                'date': date.today().strftime('%Y-%m-%d'),
                'passed': 'YES',
                'problems': 'False',
                'property1_value': 'B.masic',
                'property1_key': 'OPERATOR',
                'property2_value': 'INSTRUMENT',
                'property2_key': '',
                'property3_value': 'ANALYSIS_VERSION',
                'property3_key': '',
                'result1_key': 'VIN_DROP',
                'result1_value': data['Vin Drop (V)'],
                'result2_key': 'GND_DROP',
                'result2_value': data['GND Drop (V)'],
                'result3_key': 'EFFECTIVE RESISTANCE',
                'result3_value': data['Total Resistance(mOhms)'],
                'result4_key': 'HV_LEAKAGE',
                'result4_value': data['Capacitor leakage test Last (V)'],
                'result5_key': 'LEAKAGE_CURRENT (nA)',
                'result5_value': data['Capacitor Equivalent leakage current(nA)'],
                'result6_key': 'NTC_VOLTAGE',
                'result6_value': data['NTC Last (V)'],
                'result7_key': 'NTC_VALUE',
                'result7_value': data['NTC value (Kohms)']
            })
            
            all_data.append(result_data)
    
    if all_data:
        final_result = pd.concat(all_data, ignore_index=True)
        final_result.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")

if __name__ == "__main__":
    import os
    cwd = os.getcwd()
    input_folder = "C:/Users/User/Documents/GitHub/picoLogger/picolog_folder/"
    output_file = "C:/Users/User/Documents/GitHub/picoLogger/output_file.csv"
    perform_calculations(input_folder, output_file)


Calculation completed. Output saved to C:/Users/User/Documents/GitHub/picoLogger/output_file.csv.
